### 📊 EDA Report — Классификация типов личности


###  Описание данных
- **Размер**: 20 000 строк, 30 столбцов  
- **Таргет**: `personality_type` (3 класса: Introvert / Extrovert / Ambivert)
- **Признаки**: 29 числовых шкал (0–10)


###  Стартовый аудит качества данных (Data Quality Check)
**Проверки**:
- пропуски: **не обнаружены**
- дубликаты строк: **не обнаружены**
- типы данных: признаки числовые (`float`), таргет категориальный (`object`)
- диапазоны: значения признаков находятся в шкале **0–10** (по сводной статистике max=10, min чаще 0)

**Вывод**: данные чистые, **агрессивная очистка не требуется**


###  Баланс классов 
- 3 класса распределены **примерно равномерно** (≈33.3% каждый)
**Распределение классов**
![Распределение классов](../../../reports/figures/pieplot_class_distribution.png)



###  Описательная статистика признаков 
По большинству признаков:
- среднее и медиана близки → нет сильной асимметрии
- skewness близок к 0 → распределения в целом не перекошены
- kurtosis отрицательная → распределения более “плоские”, что типично для ограниченных шкал и/или синтетических данных

**Вывод**: распределения стабильные, явных “сломанных” шкал не выявлено.
Подробные значения можно посмотреть в таблице "reports/data/summary.csv"

### Распределения признаков по классам (унивариантный сигнал)
Визуальный анализ (гистограммы/boxplot по классам) выявил признаки:

**Признаки с сильным унивариантным сигналом**:
- `party_liking`
- `alone_time_preference`
- `public_speaking_comfort`
- `talkativeness`
- `excitement_seeking`

Наблюдение: медианы **Introvert** и **Extrovert** разнесены (примерно 2–3 vs 7–8), а **Ambivert** чаще находится ближе к центру шкалы.

**Признаки со слабым унивариантным сигналом**:
- `empathy`
- `curiosity`
- `emotional_stability`
- `stress_handling`
- `creativity`

Наблюдение: распределения по классам сильно перекрываются; явной границы по одному признаку не видно.

![Распределение creativity по классам](../../../reports/figures/histplot_creativity.png)



![Распределение party_liking по классам](../../../reports/figures/histplot_party_liking.png)

###  Выбросы и аномалии 
- Для “сильных” признаков экстремальные значения могут быть **информативными** (разделение полюсов Introvert↔Extrovert).
- Для “слабых” признаков наблюдались хвосты к нижней границе шкалы, **без заметного накопления на 0**, что больше похоже на “естественный хвост”, а не на артефакт пропусков.

**Вывод**: явных проблем качества в виде массовых аномалий не выявлено; выбросы не требуют агрессивной фильтрации до моделирования.


![Box Plot of party_liking by Personality Type](../../../reports/figures/boxplot_party_liking.png)


![Box Plot of creativity by Personality Type](../../../reports/figures/boxplot_creativity.png)

###  Взаимосвязи признаков (корреляции, редундантность, латентные кластеры)
Матрица корреляций показала **двухполюсную структуру**:

**Социально-экстравертный кластер (высокие положительные корреляции)**:
- `party_liking` ↔ `public_speaking_comfort`: **0.706**
- `party_liking` ↔ `talkativeness`: **0.699**
- `party_liking` ↔ `excitement_seeking`: **0.699**
- `party_liking` ↔ `social_energy`: **0.696**
- `talkativeness` ↔ `public_speaking_comfort`: **0.661**

**Интровертный полюс (сильные отрицательные корреляции)**:
- `party_liking` ↔ `alone_time_preference`: **−0.703**
- `public_speaking_comfort` ↔ `alone_time_preference`: **−0.663**
- `excitement_seeking` ↔ `alone_time_preference`: **−0.659**
- `talkativeness` ↔ `alone_time_preference`: **−0.658**
- `social_energy` ↔ `alone_time_preference`: **−0.655**

**Вывод**:
- признаки “социального кластера” частично **редундантны** (измеряют близкую латентную черту),
- `alone_time_preference` выступает как “противоположная ось” к социальному кластеру.

![Heatmap корреляций](../../../reports/figures/Corr_heatmap.png)



###  Анализ пар признаков (2D scatter)
Наблюдения:
- для некоторых пар “сильных” признаков разделение классов идёт **по диагонали** → важна комбинация признаков, а не один признак
- для пары “сильный + слабый” (пример: `party_liking` vs `creativity`) разделение выглядит **горизонтальным/вертикальным** → вклад в разделение чаще даёт один доминирующий признак

**Гипотеза по ошибкам модели**:
- наибольшая путаница ожидается между **Ambivert↔Introvert** и **Ambivert↔Extrovert**, т.к. Ambivert имеет много “пограничных” точек.

![2dscatter](../../../reports/figures/sccaterplot_party_likingvstalkativeness.png)

![2dscatter](../../../reports/figures/sccaterplot_public_speaking_comfortvsempathy.png)


![2dscatter](../../../reports/figures/sccaterplot_creativityvsempathy.png)

###  PCA (снижение размерности для визуализации)
Результаты:
- для сохранения **95% дисперсии** требуется **25 компонент** → пространство признаков остаётся многомерным, “сжимаемость” ограничена
- при этом на **scatter PCA** (PC1–PC2) наблюдается **чёткое разделение 3 классов** 
**Корректная интерпретация**:
- PCA не оптимизирует разделение классов, но если классы хорошо разделяются уже в первых компонентах, значит в данных есть сильные совместные направления вариативности, связанные с классами.
- При этом 25 компонент на 95% означает, что помимо “главной оси” есть ещё много дополнительных вариаций.

![2dscatter](../../../reports/figures/LDA.png)


### Профили типов личности (Radar charts по тематическим кластерам)
Radar chart по средним значениям по классам помогает “портретировать” типы:
- Extrovert выше на социальных шкалах (`party_liking`, `talkativeness`, `public_speaking_comfort`, `social_energy`)
- Introvert выше на `alone_time_preference`
- Ambivert чаще в середине, но по некоторым осям может быть ближе к одному из полюсов:
  - ближе к Introvert: `listening_skill`, `reading_habit`
  - ближе к Extrovert: `work_style_collaborative`

**Важно**: radar показывает средние значение


![radar](../../../reports/figures/radar1.png)

![radar](../../../reports/figures/radar2.png)

![radar](../../../reports/figures/radar3.png)

### Итоговые выводы
- Данные чистые: пропуски и дубликаты отсутствуют, шкалы признаков валидны (0–10).
- Классы сбалансированы (≈33% каждый), что снижает риск перекоса модели из-за дисбаланса.
- Есть выраженная “ось социальной активности”: признаки соц.кластера коррелируют между собой и противопоставлены `alone_time_preference`.
- Признаки делятся на “сильные” (социальные) и “слабые” (например, эмоциональная устойчивость/креативность), где распределения по классам перекрываются.
- Ambivert часто является промежуточным классом и имеет “пограничные” области с другими классами; в one-vs-rest анализе отличия Ambivert выражены скорее через структуру распределения (разброс), чем через большой сдвиг медианы.
- PCA показывает, что для 95% дисперсии нужно 25 компонент, но при этом в PC1–PC2 визуально классы могут разделяться достаточно хорошо.